### Ejercicio 2

Julián Cámara Miró

Para probar, vamos a definir una función que aplique n rotaciones a un unico qubit, cada una de pi/2**i rotaciones.

In [26]:
import cirq
def n_rotaciones(circuit: cirq.Circuit, qubit: cirq.NamedQubit, n: int):

    for i in range(1, n+1):
        circuit.append(cirq.Z(qubit)**(0.5**i))

    return circuit

Probamos la función anterior. Vamos a probar con 2 rotaciones a un qubit de valor inicial 1. Si funciona correctamente, se debería obtener el siguiente resultado: 

$\begin{pmatrix} 1  & 0 \\ 0 & e^{π/2} \\ \end{pmatrix} \Bigg(  \begin{pmatrix} 1  & 0 \\ 0 & e^{π/4}  \\ \end{pmatrix} \big| 1 \big \rangle \Bigg) = \begin{pmatrix} 0 \\-sin(π/4) + cos(π/4) \\ \end{pmatrix}
$

In [29]:
import numpy as np
np.set_printoptions(precision=2, suppress=False)

qubit = cirq.NamedQubit('Prueba')
circuit = cirq.Circuit()
circuit.append(cirq.X(qubit))
circuit = n_rotaciones(circuit, qubit, 2)

print(circuit)

sim = cirq.Simulator()
vector_final = sim.simulate(circuit).final_state_vector
print('Resultado:', vector_final)

Prueba: ───X───S───T───
Resultado: [ 0.  +0.j   -0.71+0.71j]


Vemos que el resultado coincide con el esperado. Ahora vamos a crear una función que nos cree el circuito de la QFT para n qubits.

In [72]:
# Función que nos crea el circuito de la QFT.
# Los qubits se inicializan al valor proporcionado por el argumento 'ini'
def QFT(N: int, ini: int):

    qubits = cirq.LineQubit.range(N)
    circuit = cirq.Circuit()

    for i in range(N):
        if (ini >> i & 0x1) == 1:
            circuit.append(cirq.X(qubits[i]))

    for i in range(N):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i + 1, N):
            circuit.append(cirq.CZ(qubits[j], qubits[i])**(0.5**(j-i)))

    return circuit

print(QFT(3, 0b001))

                      ┌───────┐
0: ───X───H───@────────@──────────────────────
              │        │
1: ───────────@^0.5────┼─────H────@───────────
                       │          │
2: ────────────────────@^0.25─────@^0.5───H───
                      └───────┘


In [73]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.1f}'.format})

circuit = QFT(4, 0b1101)
print(circuit)

sim = cirq.Simulator()
vector_final = sim.simulate(circuit).final_state_vector
print('Resultado:', vector_final)

                      ┌───────┐   ┌────────────┐   ┌───────┐
0: ───X───H───@────────@───────────@───────────────────────────────────────
              │        │           │
1: ───────────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                       │           │      │         │
2: ───X────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                                   │                │          │
3: ───X────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                      └───────┘   └────────────┘   └───────┘
Resultado: [ 0.25+0.j   -0.25+0.j    0.  -0.25j  0.  +0.25j -0.18+0.18j  0.18-0.18j
  0.18+0.18j -0.18-0.18j -0.1 -0.23j  0.1 +0.23j -0.23+0.1j   0.23-0.1j
  0.23+0.1j  -0.23-0.1j   0.1 -0.23j -0.1 +0.23j]
